In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
PyTorch 机器信息一键速查
> python torch_sysinfo.py
"""
import os
import sys
import platform
import subprocess
import torch
import psutil   # pip install psutil

# ---------- 颜色工具 ----------
class Color:
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKCYAN = "\033[96m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"

def cprint(text, color=Color.OKGREEN):
    print(color + text + Color.ENDC)

# ---------- 软件栈 ----------
cprint("==== Software Stack ====", Color.HEADER)
print(f"Python     : {platform.python_version()}")
print(f"PyTorch    : {torch.__version__}")
print(f"CUDA Build : {torch.version.cuda}")
print(f"cuDNN      : {torch.backends.cudnn.version()}")

# ---------- CPU ----------
cprint("==== CPU ====", Color.HEADER)
cpu = platform.processor() or "Unknown"
cores = psutil.cpu_count(logical=False)
threads = psutil.cpu_count(logical=True)
print(f"CPU        : {cpu}")
print(f"Cores      : {cores}")
print(f"Threads    : {threads}")

# ---------- GPU ----------
cprint("==== GPU ====", Color.HEADER)
if not torch.cuda.is_available():
    cprint("No GPU detected.", Color.FAIL)
else:
    n_gpu = torch.cuda.device_count()
    print(f"GPU Count  : {n_gpu}")
    for i in range(n_gpu):
        prop = torch.cuda.get_device_properties(i)
        print(f"  GPU {i}  : {prop.name}")
        print(f"         Memory : {prop.total_memory // 1024**2} MB")
        print(f"         Compute: {prop.major}.{prop.minor}")
    # 实时利用率
    cprint("---- Real-Time Utilization ----", Color.OKCYAN)
    try:
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=index,name,memory.used,memory.total,utilization.gpu",
             "--format=csv,noheader,nounits"],
            capture_output=True, text=True, check=True
        )
        for line in result.stdout.strip().split("\n"):
            idx, name, used, total, util = line.split(", ")
            print(f"  GPU {idx}: {util}% | Memory {used}/{total} MB")
    except FileNotFoundError:
        cprint("nvidia-smi not found; skip real-time util.", Color.WARNING)

# ---------- 分布式环境 ----------
cprint("==== Distributed / Env ====", Color.HEADER)
dist_env_keys = ["RANK", "LOCAL_RANK", "WORLD_SIZE",
                 "MASTER_ADDR", "MASTER_PORT", "CUDA_VISIBLE_DEVICES"]
for k in dist_env_keys:
    print(f"{k:20} : {os.environ.get(k, 'Not set')}")

# ---------- 随机种子 ----------
cprint("==== Reproducibility ====", Color.HEADER)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
print("Random seed set to 42 for CPU & all GPUs.")

# ---------- 设备选择 ----------
cprint("==== Default Device ====", Color.HEADER)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.rand(1, 1).to(device)
print(f"Default tensor placed on : {x.device}")

AttributeError: module 'numpy' has no attribute 'ndarray'